<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/Script_Clase_7_Supervisado_y_No_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Inteligencia Artificial Aplicada para la Economía
## Universidad de los Andes

### 👨‍🏫 Profesores
- **Profesor Magistral:** [Camilo Vega Barbosa](https://www.linkedin.com/in/camilo-vega-169084b1/)
- **Asistente de Docencia:** [Sergio Julian Zona Moreno](https://www.linkedin.com/in/sergio-julian-zona-moreno/)

## 📊 Script Algoritmos de Machine Learning Supervisados y No Supervisados

Este script implementa los principales algoritmos de ML supervisados y no supervisados:
- Regresión Logística (con y sin regularización)
- KNN y SVM
- Árboles de Decisión, Random Forest y XGBoost
- K-means y PCA

### Requisitos de Software:
- Conocimientos básicos de Python
- Familiaridad con NumPy y Pandas
- Comprensión básica de conceptos estadísticos

### Requisitos Técnicos:
- **Token de Hugging Face**: Necesario para acceder al dataset. Puedes obtener tu token en [Hugging Face](https://huggingface.co/settings/tokens)
- **Entorno de Ejecución**:
 - Recomendado: GPU T4 (Cambiar en: Runtime -> Change runtime type -> GPU)
 - Alternativa: CPU (el código funcionará, pero será más lento)
- **Memoria RAM**: Mínimo 4GB recomendados
- **Espacio en Disco**: ~200 GB para datasets y modelos

💡 **Nota**: Aunque recomendamos usar GPU para mayor velocidad, todo el código es compatible con CPU y funcionará correctamente, solo que tomará más tiempo en ejecutarse.

## Algoritmos Supervisados

In [ ]:
# 0. Instalaciones
!pip install -q pandas numpy scikit-learn xgboost matplotlib seaborn datasets

# 1. Importaciones necesarias
import pandas as pd
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (confusion_matrix, roc_curve, roc_auc_score,
                          silhouette_score, classification_report)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import xgboost as xgb

# 2. Carga de datos
"""
DATASET: Default of Credit Card Clients
Link: https://huggingface.co/datasets/scikit-learn/credit-card-clients

Descripción: Contiene información sobre incumplimientos de pago de clientes de tarjetas
de crédito en Taiwán (abril-septiembre 2005). Incluye 23 variables explicativas y
1 variable objetivo binaria (default.payment.next.month).

Variables principales:
- LIMIT_BAL: Monto de crédito otorgado (NT dollars)
- SEX: Género (1=masculino, 2=femenino)
- EDUCATION: Nivel educativo (1=posgrado, 2=universidad, 3=secundaria, 4=otros)
- MARRIAGE: Estado civil (1=casado, 2=soltero, 3=otros)
- AGE: Edad en años
- PAY_0 a PAY_6: Estado de pago mensual (-1=pago a tiempo, 1=retraso 1 mes, etc.)
- BILL_AMT1-6: Monto facturado mensual
- PAY_AMT1-6: Monto pagado mensual
- default.payment.next.month: Variable objetivo (1=default, 0=no default)

Total de registros disponibles: 30,000
Registros utilizados en este script: 10,000 (para eficiencia computacional)
"""

# Dataset principal para algoritmos supervisados
dataset = load_dataset("scikit-learn/credit-card-clients", streaming=True)
df_credit = pd.DataFrame(list(dataset['train'].shuffle(seed=42).take(10000)))

# Para usar tu propio CSV, descomenta las siguientes líneas:
# df_credit = pd.read_csv('tu_archivo.csv')
# Asegúrate de que tenga una columna objetivo llamada 'target' o ajusta el código

print("Información del dataset cargado:")
print(f"Dimensiones: {df_credit.shape}")
print(f"Columnas: {list(df_credit.columns)}")
print(f"\nDistribución de la variable objetivo:")
print(df_credit['default.payment.next.month'].value_counts(normalize=True))

# 3. Preprocesamiento de datos
def preprocess_credit_data(df):
    """
    Preprocesa los datos de crédito para que sean aptos para machine learning.
    Los algoritmos de ML requieren que todos los datos sean numéricos y estén en escalas similares.
    """

    # PASO 1: Identificar qué variables son numéricas vs categóricas
    # Numéricas: son cantidades medibles (dinero, edad) que tienen sentido matemático
    # - LIMIT_BAL: límite de crédito en dólares
    # - AGE: edad en años
    # - BILL_AMT1-6: montos facturados cada mes
    # - PAY_AMT1-6: montos pagados cada mes
    numeric_features = ['LIMIT_BAL', 'AGE'] + [f'BILL_AMT{i}' for i in range(1,7)] + [f'PAY_AMT{i}' for i in range(1,7)]

    # Categóricas: son etiquetas o categorías sin sentido matemático directo
    # - SEX: 1=hombre, 2=mujer (no tiene sentido sumar o restar estos números)
    # - EDUCATION: 1=posgrado, 2=universidad, etc. (no es que posgrado sea "menor" que universidad)
    # - MARRIAGE: estado civil
    # - PAY_0 a PAY_6: estado de pago (-1=a tiempo, 1=1 mes tarde, etc.)
    categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE'] + ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

    # PASO 2: Limpiar datos con valores inconsistentes
    df = df.copy()  # Evitar modificar el DataFrame original
    # En EDUCATION: 0, 5, 6 no están definidos en la documentación, los agrupamos como "otros" (4)
    df['EDUCATION'] = df['EDUCATION'].replace([0, 5, 6], 4)
    # En MARRIAGE: 0 no está definido, lo agrupamos como "otros" (3)
    df['MARRIAGE'] = df['MARRIAGE'].replace([0], 3)

    # PASO 3: Estandarizar variables numéricas (media=0, desviación=1)
    # ¿Por qué? Los algoritmos son sensibles a la escala. Sin esto:
    # - LIMIT_BAL (miles/millones) dominaría sobre AGE (decenas)
    # - El modelo daría más importancia a variables con valores grandes
    scaler = StandardScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])

    # PASO 4: Codificar variables categóricas
    # Los algoritmos solo entienden números, no etiquetas
    # LabelEncoder convierte categorías a números (ej: masculino=0, femenino=1)
    # Nota: esto asume que no hay orden en las categorías
    for feature in categorical_features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature].astype(str))

    # PASO 5: Separar predictores (X) de variable objetivo (y)
    # X: todas las características que usaremos para predecir
    # y: lo que queremos predecir (si habrá default o no)
    X = df.drop('default.payment.next.month', axis=1)
    y = df['default.payment.next.month']

    # Retornamos también el scaler por si necesitamos procesar nuevos datos
    return X, y, scaler

# Procesar datos
X, y, scaler = preprocess_credit_data(df_credit)

# Verificar el resultado
print("\nDatos después del preprocesamiento:")
print(f"Forma de X: {X.shape}")
print(f"Primeras 3 columnas de X:\n{X.iloc[:5, :3]}")  # Muestra que ahora todo es numérico
print(f"\nDistribución de y: {y.value_counts()}")

# 4. División de datos
# test_size: proporción de datos para prueba (0.2 = 20%)
# random_state: semilla para reproducibilidad
# stratify: mantiene la proporción de clases en train y test
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTamaño conjunto entrenamiento: {X_train.shape}")
print(f"Tamaño conjunto prueba: {X_test.shape}")
print(f"Distribución de clases en entrenamiento: {pd.Series(y_train).value_counts(normalize=True).round(3).to_dict()}")

# 5. Definición de modelos supervisados
models = {
   # Regresión Logística sin regularización
   'Logit_Basico': LogisticRegression(
       random_state=42,
       max_iter=1000  # Número máximo de iteraciones
   ),

   # Regresión Logística con regularización L2 (Ridge)
   'Logit_Ridge': LogisticRegression(
       penalty='l2',      # Tipo de regularización
       C=1.0,            # Inverso de la fuerza de regularización (menor C = más regularización)
       random_state=42,
       max_iter=1000
   ),

   # Regresión Logística con regularización L1 (Lasso)
   'Logit_Lasso': LogisticRegression(
       penalty='l1',
       C=1.0,            # Inverso de la fuerza de regularización
       solver='liblinear',  # Necesario para L1
       random_state=42,
       max_iter=1000
   ),

   # K-Nearest Neighbors
   'KNN': KNeighborsClassifier(
       n_neighbors=5,     # Número de vecinos a considerar (más vecinos = decisión más suave)
       weights='uniform', # 'uniform' o 'distance' (distance da más peso a vecinos cercanos)
       metric='euclidean' # Métrica de distancia
   ),

   # Support Vector Machine
   'SVM': SVC(
       kernel='rbf',      # Tipo de kernel: 'linear', 'poly', 'rbf', 'sigmoid'
       C=1.0,            # Parámetro de regularización (mayor C = menos regularización)
       gamma='scale',     # Coeficiente del kernel (afecta la influencia de cada punto)
       probability=True,  # Necesario para obtener probabilidades
       random_state=42
   ),

   # Árbol de Decisión
   'DecisionTree': DecisionTreeClassifier(
       criterion='entropy',    # Criterio de división: 'gini' o 'entropy'
       max_depth=6,           # Profundidad máxima (menor = menos overfitting)
       min_samples_split=20,  # Mínimo de muestras para dividir un nodo
       min_samples_leaf=10,   # Mínimo de muestras en una hoja
       random_state=42
   ),

   # Random Forest
   'RandomForest': RandomForestClassifier(
       n_estimators=100,      # Número de árboles (más árboles = mejor pero más lento)
       max_depth=None,        # Profundidad máxima de cada árbol
       min_samples_leaf=5,    # Mínimo de muestras en hojas
       max_features='sqrt',   # Número de features a considerar en cada split
       n_jobs=-1,            # Usar todos los cores disponibles
       random_state=42
   ),

   # XGBoost
   'XGBoost': xgb.XGBClassifier(
       learning_rate=0.1,     # Tasa de aprendizaje (menor = más conservador)
       n_estimators=100,      # Número de árboles
       max_depth=6,          # Profundidad máxima de cada árbol
       subsample=0.8,        # Fracción de muestras para entrenar cada árbol
       colsample_bytree=0.8, # Fracción de columnas para cada árbol
       random_state=42
   )
}

# 6. Entrenamiento y predicciones
predictions = {}
probabilities = {}

for name, model in models.items():
   print(f"Entrenando {name}...")
   model.fit(X_train, y_train)
   predictions[name] = model.predict(X_test)
   if hasattr(model, 'predict_proba'):
       probabilities[name] = model.predict_proba(X_test)[:, 1]
   else:
       probabilities[name] = model.decision_function(X_test)

# 7. Evaluación - Métricas tradicionales
def plot_confusion_matrices(y_true, predictions_dict):
   n_models = len(predictions_dict)
   fig, axes = plt.subplots(2, 4, figsize=(20, 10))
   axes = axes.ravel()

   for idx, (name, y_pred) in enumerate(predictions_dict.items()):
       cm = confusion_matrix(y_true, y_pred)
       sns.heatmap(cm, annot=True, fmt='d', ax=axes[idx], cmap='Blues')
       axes[idx].set_title(f'{name}')
       axes[idx].set_xlabel('Predicción')
       axes[idx].set_ylabel('Real')

   # Ocultar subplots vacíos si hay menos de 8 modelos
   for idx in range(n_models, 8):
       axes[idx].axis('off')

   plt.tight_layout()
   plt.show()

def plot_roc_curves(y_true, probabilities_dict):
   plt.figure(figsize=(10, 8))

   for name, probs in probabilities_dict.items():
       fpr, tpr, _ = roc_curve(y_true, probs)
       auc = roc_auc_score(y_true, probs)
       plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})')

   plt.plot([0, 1], [0, 1], 'k--', label='Random')
   plt.xlabel('Tasa de Falsos Positivos')
   plt.ylabel('Tasa de Verdaderos Positivos')
   plt.title('Curvas ROC - Comparación de Modelos')
   plt.legend()
   plt.grid(True, alpha=0.3)
   plt.show()

# Visualizar resultados
plot_confusion_matrices(y_test, predictions)
plot_roc_curves(y_test, probabilities)

# Tabla comparativa de métricas
print("\nMétricas de evaluación:")
print("-" * 80)
print(f"{'Modelo':<15} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'AUC-ROC':<10}")
print("-" * 80)

for name in predictions.keys():
   report = classification_report(y_test, predictions[name], output_dict=True)
   auc = roc_auc_score(y_test, probabilities[name])

   print(f"{name:<15} "
         f"{report['accuracy']:<10.3f} "
         f"{report['1']['precision']:<10.3f} "
         f"{report['1']['recall']:<10.3f} "
         f"{report['1']['f1-score']:<10.3f} "
         f"{auc:<10.3f}")

# 8. Validación cruzada
# Usar F1-score como métrica principal
cv_scores = {}
cv_scores_stratified = {}

# K-fold normal
kfold = 5
for name, model in models.items():
   scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='f1')
   cv_scores[name] = scores

# K-fold estratificado
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for name, model in models.items():
   scores = cross_val_score(model, X_train, y_train, cv=skfold, scoring='f1')
   cv_scores_stratified[name] = scores

# Visualizar resultados de CV
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# CV normal
data_cv = [cv_scores[name] for name in models.keys()]
ax1.boxplot(data_cv, labels=models.keys())
ax1.set_title('Validación Cruzada K-Fold (F1-Score)')
ax1.set_xlabel('Modelo')
ax1.set_ylabel('F1-Score')
ax1.tick_params(axis='x', rotation=45)

# CV estratificado
data_cv_strat = [cv_scores_stratified[name] for name in models.keys()]
ax2.boxplot(data_cv_strat, labels=models.keys())
ax2.set_title('Validación Cruzada Estratificada (F1-Score)')
ax2.set_xlabel('Modelo')
ax2.set_ylabel('F1-Score')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Imprimir medias y desviaciones
print("\nResultados de Validación Cruzada (F1-Score):")
print("-" * 60)
print(f"{'Modelo':<15} {'CV Normal (media±std)':<25} {'CV Estratificado (media±std)':<25}")
print("-" * 60)
for name in models.keys():
   print(f"{name:<15} "
         f"{np.mean(cv_scores[name]):.3f}±{np.std(cv_scores[name]):.3f}"
         f"{'':>10}"
         f"{np.mean(cv_scores_stratified[name]):.3f}±{np.std(cv_scores_stratified[name]):.3f}")

# Observa: ¿Qué modelos tienen menor variabilidad? ¿Cuáles mantienen mejor rendimiento?
# La CV estratificada suele dar resultados más estables en datasets desbalanceados



## Algoritmos Supervisados

In [ ]:
# 9. Algoritmos no supervisados
"""
Para algoritmos no supervisados, utilizamos el mismo dataset pero con un enfoque diferente.
Aquí no usamos la variable objetivo, solo exploramos patrones naturales en los datos.

El clustering puede revelar:
- Grupos naturales de clientes con comportamientos similares
- Segmentos de riesgo no evidentes en el análisis supervisado
- Variables que dominan la estructura de los datos
"""

# Cargar dataset para clustering
dataset_clustering = load_dataset("scikit-learn/credit-card-clients", streaming=True)
df_clustering = pd.DataFrame(list(dataset_clustering['train'].shuffle(seed=42).take(10000)))

# Seleccionar solo características numéricas y estandarizar
numeric_cols = df_clustering.select_dtypes(include=['float64', 'int64']).columns
X_clustering = df_clustering[numeric_cols]
X_clustering_scaled = StandardScaler().fit_transform(X_clustering)

print(f"\nDatos para clustering:")
print(f"Dimensiones: {X_clustering_scaled.shape}")
print(f"Variables utilizadas: {list(numeric_cols)}")

# K-means con diferentes números de clusters
inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(
        n_clusters=k,           # Número de clusters
        init='k-means++',       # Método de inicialización ('k-means++' o 'random')
        n_init=10,             # Número de veces que se ejecuta con diferentes centroides
        max_iter=300,          # Máximo de iteraciones
        random_state=42
    )
    kmeans.fit(X_clustering_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_clustering_scaled, kmeans.labels_))

# Visualizar método del codo y silhouette score
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(k_range, inertias, marker='o')
ax1.set_title('Método del Codo')
ax1.set_xlabel('Número de clusters (k)')
ax1.set_ylabel('Inercia')
ax1.grid(True)

ax2.plot(k_range, silhouette_scores, marker='o', color='orange')
ax2.set_title('Silhouette Score')
ax2.set_xlabel('Número de clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Observa: El codo indica donde agregar más clusters no mejora mucho
# Silhouette score más alto indica mejor separación de clusters

# Determinar el número óptimo de clusters
# Basado en el silhouette score más alto
optimal_k_silhouette = k_range[np.argmax(silhouette_scores)]
print(f"\nNúmero óptimo de clusters según Silhouette Score: {optimal_k_silhouette}")
print(f"Silhouette Score máximo: {max(silhouette_scores):.3f}")

# Para el análisis posterior, usar k=3 o el óptimo encontrado
k_final = 3  # Puedes cambiar esto a optimal_k_silhouette si prefieres
print(f"\nNúmero de clusters elegido para el análisis: {k_final}")

# PCA para reducción de dimensionalidad
# Aplicar PCA
pca_full = PCA(random_state=42)
pca_full.fit(X_clustering_scaled)

# Visualizar varianza explicada
cumsum_var = np.cumsum(pca_full.explained_variance_ratio_)
n_components_95 = np.argmax(cumsum_var >= 0.95) + 1

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumsum_var) + 1), cumsum_var, marker='o')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% varianza')
plt.axvline(x=n_components_95, color='g', linestyle='--',
            label=f'{n_components_95} componentes')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('PCA - Varianza Explicada')
plt.legend()
plt.grid(True)
plt.show()

print(f"\nComponentes necesarios para 95% de varianza: {n_components_95}")
print(f"Reducción de dimensionalidad: de {X_clustering_scaled.shape[1]} a {n_components_95} variables")

# Aplicar PCA con 2 componentes para visualización
pca_2d = PCA(n_components=2, random_state=42)
X_pca_2d = pca_2d.fit_transform(X_clustering_scaled)

# K-means en datos con PCA usando el número de clusters elegido
kmeans_pca = KMeans(n_clusters=k_final, random_state=42)
labels_pca = kmeans_pca.fit_predict(X_pca_2d)

# Visualizar clusters
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=labels_pca, cmap='viridis')
plt.xlabel('Primera componente principal')
plt.ylabel('Segunda componente principal')
plt.title(f'Clusters K-means en espacio PCA (k={k_final})')
plt.colorbar(scatter, label='Cluster')
plt.show()

# Comparar K-means con y sin PCA
kmeans_original = KMeans(n_clusters=k_final, random_state=42)
labels_original = kmeans_original.fit_predict(X_clustering_scaled)

print(f"\nComparación K-means con k={k_final}:")
print(f"Silhouette Score (datos originales): {silhouette_score(X_clustering_scaled, labels_original):.3f}")
print(f"Silhouette Score (con PCA 2D): {silhouette_score(X_pca_2d, labels_pca):.3f}")

# Observa: PCA puede mejorar o empeorar el clustering dependiendo de los datos
# Un silhouette score más alto indica clusters mejor definidos